### **Cài đặt những thư viện cần thiết**

In [ ]:
!pip install langchain langchain-community langchain-huggingface chromadb pymupdf transformers accelerate torch faiss-cpu networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.llms import Ollama
import torch
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from langchain.schema import Document
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")


In [ ]:
loader = PyMuPDFLoader("/content/doc1.pdf")

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""]
)
chunked_documents = text_splitter.split_documents(documents)
db1 = Chroma.from_documents(chunked_documents, embeddings)

In [ ]:
loader = PyMuPDFLoader("/content/doc2.pdf")

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""]
)
chunked_documents = text_splitter.split_documents(documents)
db2 = Chroma.from_documents(chunked_documents, embeddings)

In [ ]:
loader = PyMuPDFLoader("/content/doc3.pdf")

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""]
)
chunked_documents = text_splitter.split_documents(documents)
db3 = Chroma.from_documents(chunked_documents, embeddings)

In [ ]:
login("YOUR_HUGGING_FACE_KEY")
def load_llm(model_file):
    # Load model và tokenizer
    model = AutoModelForCausalLM.from_pretrained(
        model_file,
        device_map="auto",
        torch_dtype=torch.float16,
        trust_remote_code=True
    )

    tokenizer = AutoTokenizer.from_pretrained(
        model_file,
        trust_remote_code=True
    )



    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=5000,
        temperature=0.1,
        device_map="auto"
    )

    llm = HuggingFacePipeline(pipeline=pipe)

    return llm

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

retriever1 = db1.as_retriever(search_kwargs={"k": 4})
retriever2 = db2.as_retriever(search_kwargs={"k": 4})
retriever3 = db3.as_retriever(search_kwargs={"k": 4})

llm = load_llm("meta-llama/Llama-3.2-3B-Instruct")

rag_prompt = PromptTemplate.from_template(
  """
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an AI assistant helping to answer questions based on provided contexts

    <|eot_id|><|start_header_id|>user<|end_header_id|>

    Context: {context}
    Question: {input}

    <|eot_id|><|start_header_id|>assistant<|end_header_id|>
  """
)


rewrite_prompt = PromptTemplate.from_template(
  """
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an AI assistant helping to rewrite the given answer to make it cleaner, more structured, and easier to understand.

    <|eot_id|><|start_header_id|>user<|end_header_id|>

    answer: [
      Binary Search Trees (BSTs) are a type of binary tree where each node has a key and a value, and satisfies the restriction that the key in any node is larger than the keys in all nodes in its left subtree and smaller than the keys in all nodes in its right subtree.
    ]
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>

    rewrited answer: [
      A Binary Search Tree (BST) is a type of binary tree where each node contains a key and a value, and the tree follows a specific rule:

        1. All keys in the left subtree of a node must be less than the node’s key.
        2. All keys in the right subtree must be greater than the node’s key.
        3. This rule is applied recursively to every node in the tree.

      Because of this property, BSTs allow for efficient operations like searching, inserting, and deleting elements, usually with an average time complexity of O(log n) when the tree is balanced.
    ]

    <|eot_id|><|start_header_id|>user<|end_header_id|>

    answer: [
      The text mentions the following sorting algorithms:
        1. Counting Sort
        2. Bucket Sort
        3. Radix Sort
        4. Bubble Sort
        5. Insertion Sort
    ]
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>

    rewrited answer: [
      this is a few sorting algorithms:
        1. Counting Sort
        2. Bucket Sort
        3. Radix Sort
        4. Bubble Sort
        5. Insertion Sort
    ]


    <|eot_id|><|start_header_id|>user<|end_header_id|>

    answer: [
      {input}
    ]
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>

  """
)


rag_chain = LLMChain(llm=llm, prompt=rag_prompt)
rewrite_chain = LLMChain(llm=llm, prompt=rewrite_prompt)

In [ ]:
import torch
import re

# torch.cuda.empty_cache()
# torch.cuda.memory_summary(device=None, abbreviated=False)

def extract_answer(text):
    pattern = r"<\|eot_id\|><\|start_header_id\|>assistant<\|end_header_id\|>"
    matches = list(re.finditer(pattern, text))
    if matches:
        last_match = matches[-1]
        return text[last_match.end():].strip()
    return None


def query_rag_with_llm(rag_chain, rewrite_chain, question):
    retrieved_docs = retriever1.get_relevant_documents(question)
    retrieved_docs = [doc.page_content for doc in retrieved_docs]
    result = rag_chain.invoke({"input": question, "context": retrieved_docs})


    retrieved_docs = retriever2.get_relevant_documents(question)
    retrieved_docs = [doc.page_content for doc in retrieved_docs]
    retrieved_docs.append(extract_answer(result['text']))
    result = rag_chain.invoke({"input": question, "context": retrieved_docs})


    retrieved_docs = retriever3.get_relevant_documents(question)
    retrieved_docs = [doc.page_content for doc in retrieved_docs]
    retrieved_docs.append(extract_answer(result['text']))
    result = rag_chain.invoke({"input": question, "context": retrieved_docs})


    result = rewrite_chain.invoke({extract_answer(result['text'])})

    return extract_answer(result['text'])
question = "What are the time complexities of common operations (insertion, deletion, search) in a binary search tree (BST), and how do they compare to those in a balanced binary search tree?"
print(query_rag_with_llm(rag_chain, rewrite_chain, question))

In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok transformers

In [ ]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel
from pyngrok import conf, ngrok
from fastapi.middleware.cors import CORSMiddleware

# Bật lại event loop trong notebook
nest_asyncio.apply()

# Ngrok token
conf.get_default().auth_token = "YOUR_NGROK_KEY"

# Khởi tạo FastAPI
app = FastAPI()

# Cho phép CORS để web gửi request được
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

# Dữ liệu gửi đến
class Message(BaseModel):
    message: str

# Định nghĩa endpoint
@app.post("/chat")
def chat_api(msg: Message):
    prompt = msg.message
    print(prompt)
    output = query_rag_with_llm(rag_chain, rewrite_chain, prompt)  # Đã định nghĩa sẵn
    return {"reply": output}

# Tạo tunnel
public_url = ngrok.connect(8000)
print(f"🔗 API endpoint: {public_url}/chat")

# Chạy server
uvicorn.run(app, port=8000)
